In [13]:
import json
import re
import pandas as pd
from ranking import get_lemmas_len, get_hyponyms, get_frequencies, get_lemmas_len_avg
from sklearn.tree import DecisionTreeClassifier

## Classificatore di basicness
L'esercizio richiedeva di costruire un classificatore automatico binario per determinare se una parola è basic o avanzata.
Il data set fornito è stato usato per allenare il classificatore binario. Nella cella seguente si trova l'apertura e la lettura di uno dei file, in particolare vengono estratti i synset e i valori assegnati dall'utente sulla basicness.

In [14]:
with open('dataset_basic_advanced_TLN2023/1.json', 'r') as file:
    contenuto_json = json.load(file)

synset_target = [0 if answer == 'basic' else 1 for answer in contenuto_json['answers']]

synset_values = re.findall(r"Synset\(.*?\)", str(contenuto_json['dataset']))
synset_names = [synset_values[i].replace("Synset('", "").replace("')", "").split(".")[0] for i in range(0, len(synset_values))]

### Dati per il test
Di seguito viene creata una lista di parole su cui testare il classificatore
Come features per la classificazione sono stati usati:
- La lunghezza del primo lemma del senso più frequente
- La lunghezza media dei lemmi dei sensi
- Il numero di iponimi dei sensi
- La frequenza del primo lemma del senso più frequente

Questi dati sono tutti prelevati da Wordnet.

È stato fornito anche un vettore con lo score aspettato. Di seguito è stampato il DataFrame con i tre valori per ogni termine.

In [15]:
name_test = ['cat', 'rose', 'hello', 'serendipity', 'positivity', 'bye', 'girl', 'eccentric', 'sophisticated', 'house']
lemma_test = get_lemmas_len(name_test)
lemmi_avg_test = get_lemmas_len_avg(name_test)
hyp_test = get_hyponyms(name_test)
freq_test = get_frequencies(name_test)
y_test = [0, 0, 0, 1, 1, 0, 0, 1, 1,0]

df_test = pd.DataFrame({'Synset':name_test, 'Lunghezza lemma': lemma_test, 'Lunghezza media lemmi': lemmi_avg_test, 'Numero iponimi': hyp_test, 'Frequenza': freq_test})

df_test

,Synset,Lunghezza lemma,Lunghezza media lemmi,Numero iponimi,Frequenza
0,cat,3,7.0435,12,18
1,rose,4,5.6806,39,5
2,hello,5,6.0000,0,1
3,serendipity,11,11.0000,0,0
4,positivity,10,12.7059,2,0
5,bye,3,7.0588,0,0
6,girl,4,7.1333,33,80
7,eccentric,9,7.7143,2,3
8,sophisticated,12,10.2143,0,0
9,house,5,6.7586,69,157


### Training
Di seguito vengono calcolate le tre liste di valori per le parole estratte dal file e mostrati in tabella. Dopo diche viene creato il classificatore da quei dati.

In [16]:
lemma_train = get_lemmas_len(synset_names)
lemmi_avg_train = get_lemmas_len_avg(synset_names)
hyp_train = get_hyponyms(synset_names)
freq_train = get_frequencies(synset_names)

df = pd.DataFrame({'Synset':synset_names, 'Lunghezza lemma': lemma_train, 'Lunghezza media lemmi': lemmi_avg_train,'Numero iponimi': hyp_train, 'Frequenza': freq_train})

df

,Synset,Lunghezza lemma,Lunghezza media lemmi,Numero iponimi,Frequenza
0,war,3,5.1250,14,78
1,fiefdom,7,7.0000,0,0
2,bed,3,6.7143,47,51
3,return_on_invested_capital,26,16.3333,0,0
4,texture,7,6.6667,5,13
...,...,...,...,...,...
499,reading,7,6.6923,28,22
500,sanctimoniousness,17,13.5000,0,0
501,chalcedony,10,9.5000,7,0
502,stopcock,8,6.6667,0,0


In [17]:
classifier = DecisionTreeClassifier(random_state=0)
classifier.fit(df[['Lunghezza media lemmi', 'Numero iponimi', 'Frequenza']], synset_target)

DecisionTreeClassifier(random_state=0)

In [18]:
classifier_due = DecisionTreeClassifier(random_state=0)
classifier_due.fit(df[['Lunghezza lemma', 'Numero iponimi', 'Frequenza']], synset_target)

DecisionTreeClassifier(random_state=0)

In [19]:
v = classifier.score(df[['Lunghezza media lemmi', 'Numero iponimi', 'Frequenza']], synset_target)
print('Valore di accuratezza del risultato:', round(v, 4))

Valore di accuratezza del risultato: 0.9762


In [20]:
v_due = classifier_due.score(df[['Lunghezza lemma', 'Numero iponimi', 'Frequenza']], synset_target)
print('Valore di accuratezza del risultato:', round(v_due, 4))

Valore di accuratezza del risultato: 0.9583


### Test
Il classificatore viene usato per fare predizione sul valore di basicness dei termini di test.
Il risultato viene stampato sotto in un array di 0 e 1, inoltre nell'ultima cella viene anche calcolato lo score di accuratezza del risultato ottenuto.

In [21]:
y = classifier.predict(df_test[['Lunghezza media lemmi', 'Numero iponimi', 'Frequenza']])
y

array([0, 0, 1, 1, 1, 1, 0, 0, 1, 0])

In [22]:
v = classifier.score(df_test[['Lunghezza media lemmi', 'Numero iponimi', 'Frequenza']], y_test)
print('Valore di accuratezza del risultato:', round(v, 4))

Valore di accuratezza del risultato: 0.7


In [23]:
y_due = classifier_due.predict(df_test[['Lunghezza lemma', 'Numero iponimi', 'Frequenza']])
y

array([0, 0, 1, 1, 1, 1, 0, 0, 1, 0])

In [24]:
v = classifier_due.score(df_test[['Lunghezza lemma', 'Numero iponimi', 'Frequenza']], y_test)
print('Valore di accuratezza del risultato:', round(v, 4))

Valore di accuratezza del risultato: 0.8
